# Clustering of Cities - interactive demo
v0.1

Authors: Maria Ricci, Manuel Löhnertz (space4environment), Mohamed-Bachir Belaid (NILU)

---
### Scope
Run clustering analysis of ~700 European cities based on land, climate and socioeconomic indicators. Tune the different parameters to get the optimal result.
### Data
The collection of indicators is available ...
TODO publish data
### Requirements
TODO required packages


In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans

# for notebook interaction
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, fixed, interactive
from IPython.display import display

## Load data

Get data from database. 

Requirements: 
- `src` package installed in your conda environment. To install it, run `pip install -e .` from the root folder
- `database.ini` configuration file to connect to the database. Alternatively, set up your own data source

In [2]:
import pkg_resources
import os
from sqlalchemy import text

required = {'src'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed
if missing:
    print('src package is not installed')
else:
    from src import db_connect
    home_dir = os.environ.get('HOME')
    engine_postgresql = db_connect.create_engine(db_config = f"{home_dir}/uc1-urban-climate/database.ini")

    with engine_postgresql.begin() as conn:
        query = text("""SELECT 
                    urau_code, urau_name, _wgs84x, _wgs84y, ez_code, city_area_ha, dem_mean,
                    imd_percent_2018, treecover_percent_2018,
                    class_11100, class_11210, class_11220, class_11230,
                    class_11240, class_11300, class_12100, class_12210,
                    class_12220, class_12230, class_12300, class_12400,
                    class_13100, class_13300, class_13400, class_14100,
                    class_14200, class_21000, class_22000, class_23000,
                    class_24000, class_25000, class_31000, class_32000,
                    class_33000, class_40000, class_50000, urban_blue_percent,
                    urban_green_percent, avg_2m_temp_kelvin_2018,
                    number_of_summer_days_2018, number_of_tropical_nights_2018,
                    utci_heat_nights_2018, coastal_city, de1001v_2018, de1028v_2018,
                    de1055v_2018, ec1174v_2018, ec1010v_2018, ec1020i_2018,
                    ec3040v_2018, sa2013v_2018, de1028i_2018, de1055i_2018 
                    FROM public.city_2018_demo_view;""")
        df = pd.read_sql_query(query, conn)
    print(df.columns)

Index(['urau_code', 'urau_name', '_wgs84x', '_wgs84y', 'ez_code',
       'city_area_ha', 'dem_mean', 'imd_percent_2018',
       'treecover_percent_2018', 'class_11100', 'class_11210', 'class_11220',
       'class_11230', 'class_11240', 'class_11300', 'class_12100',
       'class_12210', 'class_12220', 'class_12230', 'class_12300',
       'class_12400', 'class_13100', 'class_13300', 'class_13400',
       'class_14100', 'class_14200', 'class_21000', 'class_22000',
       'class_23000', 'class_24000', 'class_25000', 'class_31000',
       'class_32000', 'class_33000', 'class_40000', 'class_50000',
       'urban_blue_percent', 'urban_green_percent', 'avg_2m_temp_kelvin_2018',
       'number_of_summer_days_2018', 'number_of_tropical_nights_2018',
       'utci_heat_nights_2018', 'coastal_city', 'de1001v_2018', 'de1028v_2018',
       'de1055v_2018', 'ec1174v_2018', 'ec1010v_2018', 'ec1020i_2018',
       'ec3040v_2018', 'sa2013v_2018', 'de1028i_2018', 'de1055i_2018'],
      dtype='object')


In [3]:
#TODO improve table
#TODO add link to data viz

fig = go.Figure(data=[go.Table(
    header=dict(values=["City name", "area_ha", "dem_mean", "ez_code", "imd_percent_2018", "treecover_percent_2018", "avg_2m_temp_kelvin_2018", "class_50000"]),
    cells=dict(values=[df.urau_name, df.city_area_ha, df.dem_mean, df.ez_code, df.imd_percent_2018, df.treecover_percent_2018, df.avg_2m_temp_kelvin_2018, df.class_50000]))
])

print(f'There are {df.shape[0]} cities and {df.shape[1]} indicators')
fig.show()

There are 729 cities and 53 indicators


## Data preparation
First check how many missing values there are for each feature
#TODO show variance

In [4]:
df_gaps = df.isna().sum()

fig = go.Figure(data=[go.Table(
    header=dict(values=["Attribute", "Count of NoData", "Mean", "Variance"]),
    cells=dict(values=[df.columns[5:], df_gaps[5:], df.mean(axis=0, numeric_only=True)[2:], df.var(axis=0, numeric_only=True)[2:]]))
])

fig.show()

### Treat missing values
There are two methods to treat missing values:
- `remove` the cities with missing values
- `impute` the missing values with a computed value, e.g. the average

If a feature has too many missing values, imputation does not work, and removing rows leads to a significant dataset reduction. In this case, we remove the feature


### Treat categorical features
There are three methods to deal with categorical features:
- `label_encoding` assigns a numerical value to each category. #TODO explain pro/contra
- `binary_encoding` turns each category into a new binary feature #TODO explain pro/contra
- `remove` the categorical feature altogether. May be used later for result interpretation

### Treat correlated features
TODO

In [44]:
# TODO add more info to the figure
# https://plotly.com/python/figurewidget-app/
def remove_features(df, min_cities = 100):
    df_gaps = df.isna().sum()
    columns_to_drop = df_gaps[df_gaps>min_cities].index.to_list()
    new_df = df.drop(columns=columns_to_drop)
    return new_df

def treat_missing_values(df, method='remove'):
    if method=='remove':
        new_df = df.dropna(axis='index')
    else:
        new_df = df.fillna(df.mean(numeric_only=True))
    return new_df

def treat_cat_features(df, method='binary_encoding', feature='ez_code'):
    if method=='binary_encoding':
        new_df = pd.get_dummies(df, columns=[feature])
    elif method=='label_encoding':
        le = LabelEncoder()
        le.fit(df[feature])
        new_df = df
        new_df[feature] = le.transform(df[feature])
    else:
        new_df = df.drop(columns=[feature])
    return new_df

def get_correlated_pairs(df, threshold=0.9):
    correlation_matrix = df.corr(numeric_only=True)

    # create mask to display only upper left triangle 
    mask = np.zeros_like(correlation_matrix, dtype=bool)
    mask[np.triu_indices_from(mask)] = True
    cm_abs = np.abs(correlation_matrix.mask(mask))
    correlated_attrs = cm_abs.where(cm_abs>threshold).stack()
    correlated_pairs = correlated_attrs.index.to_list()
    val1 = list(set([val for val,_ in correlated_pairs]))
    new_df = df.drop(columns=val1)
    return correlation_matrix, mask, correlated_pairs, new_df

def data_preparation(df, min_cities, missing_val_method, cat_features_method, cat_feature_name, threshold):
    df1 = remove_features(df, min_cities)
    df2 = treat_missing_values(df1, missing_val_method)
    df3 = treat_cat_features(df2, cat_features_method, cat_feature_name)
    correlation_matrix, mask, correlated_pairs, new_df = get_correlated_pairs(df3, threshold)

    return correlation_matrix, mask, correlated_pairs, new_df

# define controllers
style = {'description_width': 'initial'}
remove_features_controller = widgets.IntSlider(value=100, min=50, max=400, step=50, continuous_update=False, description="Min cities", style=style)
missing_val_controller = widgets.Dropdown(
    options=['remove', 'impute'],
    value='remove',
    description='Treat missing values in rows',
    disabled=False,
    style=style
)
categorical_controller = widgets.Dropdown(
    options=['label_encoding', 'binary_encoding', 'remove'],
    value='binary_encoding',
    description='Treat categorical features',
    disabled=False, 
    style=style
)
correlation_features_controller = widgets.FloatSlider(value=0.9, min=0.9, max=0.99, step=0.01, description="Threshold")
controller_container = widgets.VBox([remove_features_controller, missing_val_controller, categorical_controller, correlation_features_controller])
# initialize the figure
correlation_matrix, mask, correlated_pairs, new_df = data_preparation(df, 100, 'remove', 'binary_encoding', 'ez_code', 0.9)
pio.templates.default = "plotly_white"
layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.layout.Margin(l=50, r=50, b=100, t=100, pad=4),
)
fig = go.FigureWidget(
    data=go.Heatmap(
    z=correlation_matrix.mask(mask),
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale=px.colors.diverging.RdBu,
    zmin=-1,
    zmax=1,
),
layout=layout)

# bind controllers and figure
def response(change):
    df1 = remove_features(df, remove_features_controller.value)
    df2 = treat_missing_values(df1, missing_val_controller.value)
    df3 = treat_cat_features(df2, categorical_controller.value, 'ez_code')
    correlation_matrix, mask, correlated_pairs, new_df = get_correlated_pairs(df3, correlation_features_controller.value)
    with fig.batch_update():
        fig.data[0].z = correlation_matrix.mask(mask)
        fig.data[0].x = correlation_matrix.columns
        fig.data[0].y = correlation_matrix.columns

remove_features_controller.observe(response, names="value")
missing_val_controller.observe(response, names="value")
categorical_controller.observe(response, names="value")
correlation_features_controller.observe(response, names="value")
figure1 = widgets.VBox([controller_container, fig])

In [45]:
figure1

## Clustering
Once the data have been cleaned, we can start the clustering. Tune the different clustering settings to get the optimal result.
It is possible to change the following parameters:
- Feature normalization: choose between `min_max` or `normal` scaling.
- Clustering method: choose between classic `k-means clustering` or `weighted k-means clustering`. In the latter case, you must define the weights for each feature. The weights are used when updating the cluster centroids.
- Optimal number of clusters: use the elbow method to determine what is the optimal number of clusters. The figure below charts the clusters inertia versus the number of clusters. The optimal number of cluster is the one that minimizes the inertia as well as the number of clusters. The red dot on the graph indicates the current choice of number of clusters (`k`)

### WeightedKmean, revised Kmeans Algorithm to consider the weight 
The weights are considered during the update of centroids

In [54]:
#Selecting features (From 0 to 4 are cities infos, e.g.s city code)
df1 = remove_features(df, remove_features_controller.value)
df2 = treat_missing_values(df1, missing_val_controller.value)
df3 = treat_cat_features(df2, categorical_controller.value, 'ez_code')
correlation_matrix, mask, correlated_pairs, new_df = get_correlated_pairs(df3, correlation_features_controller.value)
features = new_df.iloc[:, 4:]
weight_dictionary = {
    'dem_min': 1,
    'dem_max': 1,
    'dem_std': 1,
    'dem_mean': 1,
    'ez_code': 1,
    'imd_percent_2018': 1,
    'treecover_percent_2018': 1.5,
    'city_area_ha': 1,
    'class_11100': 1,
    'class_11210': 1,
    'class_11220': 1,
    'class_11230': 1,
    'class_11240': 1,
    'class_11300': 1,
    'class_12100': 1,
    'class_12210': 1,
    'class_12220': 1,
    'class_12230': 1,
    'class_12300': 1,
    'class_12400': 1,
    'class_13100': 1,
    'class_13300': 1,
    'class_13400': 1,
    'class_14100': 1,
    'class_14200': 1,
    'class_21000': 1,
    'class_22000': 1,
    'class_23000': 1,
    'class_24000': 1,
    'class_25000': 1,
    'class_31000': 1,
    'class_32000': 1,
    'class_33000': 1,
    'class_40000': 1,
    'class_50000': 1,
    'urban_green_percent': 2,
    'urban_blue_percent': 2,
    'avg_2m_temp_kelvin_2018': 2,
    'number_of_summer_days_2018': 2,
    'number_of_tropical_nights_2018': 2,
    'utci_heat_nights_2018': 2,
    'coastal_city': 1.5,
    'de1001v_2018': 2,
    'de1028v_2018': 1.5,
    'de1055v_2018': 1.5,
    'ec1174v_2018': 1,
    'ec1010v_2018': 1,
    'ec1020i_2018': 1.5,
    'ec3040v_2018': 1,
    'sa2013v_2018': 1,
    'de1028i_2018': 1.5,
    'de1055i_2018': 1.5,
    'ez_code_ALN': 1, 
    'ez_code_ALS': 1,
    'ez_code_ARC': 1, 
    'ez_code_ATC': 1, 
    'ez_code_ATN': 1, 
    'ez_code_BOR': 1,
    'ez_code_CON': 1, 
    'ez_code_LUS': 1, 
    'ez_code_MAC': 1, 
    'ez_code_MDM': 1,
    'ez_code_MDN': 1, 
    'ez_code_MDS': 1, 
    'ez_code_NEM': 1, 
    'ez_code_PAN': 1
}
feature_weights = []
for i in features.columns:
    feature_weights.append(weight_dictionary[i])

## Normalize weights
feature_weights /= np.sum(feature_weights)

class WeightedKMeans:
    def __init__(self, n_clusters=8, max_iter=100, random_state=None):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.random_state = random_state
        self.cluster_centers_ = None
        self.labels_ = None
        self.inertia_ = None

    def fit(self, X, weights):
        np.random.seed(self.random_state)
        # Initialize cluster centers randomly
        idx = np.random.choice(X.shape[0], self.n_clusters, replace=False)
        self.cluster_centers_ = X[idx]

        for _ in range(self.max_iter):
            # Assign labels based on closest cluster center
            self.labels_ = np.argmin(np.linalg.norm(X[:, np.newaxis] - self.cluster_centers_, axis=2), axis=1)

            for i in range(self.n_clusters):
                cluster_points = X[self.labels_ == i]
                if len(cluster_points) == 0:
                    new_center_idx = np.random.randint(X.shape[0])
                    self.labels_[new_center_idx] = i
            # Update cluster centers based on the mean of the points in each cluster

            Y = X * weights
            new_centers = np.array([Y[self.labels_ == i].mean(axis=0) for i in range(self.n_clusters)])

            distances = np.linalg.norm(X - self.cluster_centers_[self.labels_], axis=1)
            self.inertia_ = np.sum(distances ** 2)
            # Check for convergence
            if np.allclose(self.cluster_centers_, new_centers):
                break

            self.cluster_centers_ = np.array(new_centers)

    def predict(self, X):
        return np.argmin(np.linalg.norm(X[:, np.newaxis] - self.cluster_centers_, axis=2), axis=1)
    


In [57]:
#TODO add select multiple widget
import warnings
warnings.filterwarnings('ignore')

k_max = 30
def scaler(features, method='min_max'):
    if method=='min_max':
        scaler_f = MinMaxScaler(feature_range=(0, 1))
    else:
        scaler_f = StandardScaler()
    normalized_features = scaler_f.fit_transform(features)
    return normalized_features, scaler_f
def clustering(normalized_features, method='normal', k=8):
    if(method=='normal'):
        kmeans = KMeans(n_init=10)
        kmeans.random_state=42
        kmeans.n_clusters = k
        kmeans.max_iter = 100
        kmeans.fit(normalized_features)
    else:
        kmeans = WeightedKMeans()
        kmeans.random_state=42
        kmeans.n_clusters = k
        kmeans.max_iter = 100
        kmeans.fit(normalized_features, feature_weights)
    return kmeans
def elbow_method(normalized_features, k_max=k_max, method='normal'):
    inertia = []
    k_range = range(1, k_max)
    if(method=='normal'):
        kmeans = KMeans(n_init=10)
        kmeans.random_state=42
        for k in k_range:
            kmeans.n_clusters = k
            kmeans.fit(normalized_features)
            inertia.append(kmeans.inertia_)
    else:
        kmeans = WeightedKMeans()
        kmeans.random_state=42
        for k in k_range:
            kmeans.n_clusters = k
            kmeans.fit(normalized_features, feature_weights)
            inertia.append(kmeans.inertia_)
    return inertia

#set up controllers
k_controller = widgets.IntSlider(value=8, min=1, max=k_max-1, step=1, description="Optimal k", continuous_update=False, style=style)
norm_method_controller = widgets.Dropdown(
    options=['min_max', 'standard'],
    value='min_max',
    description='Choose normalization method',
    disabled=False, 
    style=style
)
cluster_method_controller = widgets.Dropdown(
    options=['normal', 'weighted'],
    value='normal',
    description='Choose clustering method',
    disabled=False, 
    style=style
)
columns_controller = widgets.SelectMultiple(
    options=features.columns.tolist(),
    value=features.columns.tolist(),
    #rows=10,
    description='Columns',
    disabled=False,
    continuous_update=False, 
    style=style
)
controller_container2 = widgets.VBox([norm_method_controller, cluster_method_controller, k_controller,columns_controller])
# initialize the figure
normalized_features,fitted_scaler = scaler(features, method='min_max')
inertia = elbow_method(normalized_features, method='normal')
cluster_df = new_df[['urau_code', 'urau_name', '_wgs84x', '_wgs84y']]
cluster_df['cluster'] =  clustering(normalized_features, 
                            method=cluster_method_controller.value, 
                            k=k_controller.value).labels_
layout = go.Layout(
    autosize=False,
    width=500,
    height=500,
    margin=go.layout.Margin(l=50, r=50, b=100, t=100, pad=4),
)
fig2 = go.FigureWidget(
    data=go.Scatter(x=list(range(1, k_max)), y=inertia),
    layout=layout)
fig2.add_trace(go.Scatter(x=[8], y=[inertia[7]], mode='markers'))

# bind controllers and figure
def response2(change):
    normalized_features, fitted_scaler = scaler(features, method=norm_method_controller.value)
    inertia2 = elbow_method(normalized_features, method=cluster_method_controller.value)
    cluster_df['cluster'] =  clustering(normalized_features, 
                                method=cluster_method_controller.value, 
                                k=k_controller.value).labels_
    with fig2.batch_update():
        fig2.data[0].y = inertia2
        fig2.data[1].x = [k_controller.value]
        fig2.data[1].y = [inertia2[k_controller.value-1]]

norm_method_controller.observe(response2, names="value")
cluster_method_controller.observe(response2, names="value")
k_controller.observe(response2, names="value")
columns_controller.observe(response2, names="value")
figure2 = widgets.VBox([controller_container2, fig2])

In [58]:
figure2

## Results interpretation
### Geographical interpretation

In [59]:
from ipyleaflet import Map, GeoData
import geopandas

m = Map(center=(46.91, 7.43), zoom=5)
cmap_hex = ["#FF5733", "#C70039", "#900C3F", "#581845", "#FFC300", "#DAF7A6", "#884EA0", "#2471A3", "#2E4053", "#1B4F72", "#186A3B", "#A569BD", "#5D6D7E", "#AEB6BF", "#212F3D", "#283747", "#1B2631", "#515A5A", "#17202A", "#6E2C00"]

gdf = geopandas.GeoDataFrame(
    cluster_df, geometry=geopandas.points_from_xy(cluster_df._wgs84x, cluster_df._wgs84y))
for idx in range(0, np.max(cluster_df.cluster)+1):
    geo_data = GeoData(geo_dataframe = gdf[gdf.cluster==idx],
        point_style={'color': cmap_hex[idx], 'fillColor': cmap_hex[idx], 'radius': 5, 'fillOpacity': 0.8,'weight': 3},
        name = 'Release')
    m.add(geo_data)
m

Map(center=[46.91, 7.43], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

### Transformed feature space
How 'good' are cities separated?

In [60]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=42)
tsne_features = tsne.fit_transform(normalized_features)
# # Create a 3D scatter plot for t-SNE visualization
layout = go.Layout(
    showlegend=True,
    scene=go.Scene(
        xaxis=go.XAxis(title='t-SNE Feature 1'),
        yaxis=go.YAxis(title='t-SNE Feature 2'),
        zaxis=go.ZAxis(title='t-SNE Feature 3')
    )
)

figure3 = go.Figure(layout=layout)

for cluster_label, color in zip(range(k_controller.value), cmap_hex[:k_controller.value]):
    # Filter data points belonging to the current cluster
    cluster_indices = np.where(cluster_df.cluster == cluster_label)[0]
    figure3.add_trace(go.Scatter3d(x=tsne_features[cluster_indices, 0], y=tsne_features[cluster_indices, 1], z=tsne_features[cluster_indices, 2],
                                   mode='markers',
                                   marker=dict(size=5,
                                               color=color,
                                               opacity=0.5,)))

figure3.show()

### Cluster "profile"
#TODO get the centroids of the each cluster 
### Most influencing features
#TODO find which feature has the most influence on each cluster

In [61]:
from src.kmeans_feature_imp import KMeansInterp
kmeans_I = KMeansInterp(
        n_clusters=k_controller.value,
        ordered_feature_names=features.columns.tolist(),
        max_iter=300,
        feature_importance_method='wcss_min',  # or 'unsup2sup'
    ).fit(normalized_features)

In [64]:
fi_df = pd.DataFrame(index=new_df.columns)
for cluster in kmeans_I.feature_importances_:
    idxs, vals = kmeans_I.feature_importances_[cluster]
    tmp_df = pd.DataFrame(vals, index=idxs, columns=[cluster])
    fi_df = pd.merge(fi_df, tmp_df, left_index=True, right_index=True)

In [65]:
layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.layout.Margin(l=50, r=50, b=100, t=100, pad=4),
)
fig = go.Figure(
    data=go.Heatmap(
    z=fi_df.to_numpy(),
    x=fi_df.columns,
    y=fi_df.index,
    zmin=0,
    zmax=1,
),
layout=layout)
fig.update_xaxes(side="top")
fig.show()